# KMC simulation of PLD

In [1]:
import sys
import os
import numpy as np

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
sys.path.append('..') #/Users/l2v/work/kmcsim

Import kmc simulation tools

In [4]:
from kmcsim.buildtools import make_fcc, write_latt
from kmcsim.sim import KMCModel
from kmcsim.sim import EventTree
from kmcsim.sim import RunSim

Working directory (inputs, outputs)

In [5]:
wdir = '../data/working'

In [6]:
cat ../data/working/params

deposition: 0.1
diffusion: 0.1


Set up simulation box and initial configuration of the substrate (with atoms filling the whole box)

In [7]:
# make substrate: perfect FCC lattice of given dimensions
box = [16, 32, 4]
latt = make_fcc(box)

In [8]:
# extend the box in the z-direction to make space for new layers to grow
latt['box'][3] = 256 

In [9]:
# write initial configuration to xyz file in the working directory
write_latt(latt, os.path.join(wdir,'ni.xyz'))

In [10]:
sim = RunSim()

In [11]:
sim.read(os.path.join(wdir,'kmc.input'))

In [12]:
sim.init_sim()

FileNotFoundError: [Errno 2] No such file or directory: '../data/working/10.0'

In [ ]:
sim.run()

In [ ]:
sim.output()